In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from astra.torch.models import EfficientNet,ResNetClassifier, MLP, MLPClassifier, EfficientNetClassifier
# from astra.torch.utils import count_params
# print(count_params(EfficientNetClassifier()))
# print(count_params(ResNetClassifier()))

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch.utils.data import random_split
from torchvision.utils import make_grid
import torch.nn as nn
import copy
import time
import os
import torchvision.models as models

import argparse
from torch.utils.data import Subset
import csv
device = torch.device('cuda')
from ResNet import *
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score,precision_score,recall_score
from sklearn.metrics import ConfusionMatrixDisplay

### transforms for downstream tasks

In [4]:
# transform_test = transforms.Compose([
#         # transforms.Resize(224),
#         # transforms.ToTensor(),
#         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     ])
# s=0.5
# color_jitter = transforms.ColorJitter(0.4*4, 0.4*s, 0.4*s, 0.2*s)
# transform_eval = transforms.Compose([
#         # transforms.RandomResizedCrop((224,224), scale=(0.08, 1.0), ratio=(0.75, 1.3333333333333333)),
#         transforms.RandomHorizontalFlip(p=0.5),
#         transforms.RandomApply([color_jitter], p=0.5), 
#         transforms.RandomGrayscale(p=0.4),
#         # transforms.Resize(224),
#         # transforms.ToTensor(),
#         # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
#     ])


In [5]:
loaded_data = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/test_data.pt")
loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/new_dataset_Bangladesh_14k.pt")
images= loaded_data1['images']
labels = loaded_data1['labels']
# images = images / 255
# images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
images1 = loaded_data['images']
labels1 = loaded_data['labels']
# images1 = images1 / 255
# images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
# print(images1.shape)
# print(labels1.shape)
# print(images.shape)
# print(labels.shape)
train_images = images
train_labels = labels
test_images = images1
test_labels = labels1
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

torch.Size([14000, 3, 224, 224])
torch.Size([14000])
torch.Size([4500, 3, 224, 224])
torch.Size([4500])


In [6]:
loaded_datad1= torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/delhi_data_2250.pt")
imagesd= loaded_datad1['images']
labelsd = loaded_datad1['labels']

# images1 = images1 / 255
# images1 = (images1 print(images1.shape)
# print(labels1.shape)- images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)


In [7]:
train_images= torch.cat((train_images,imagesd),0)
train_labels= torch.cat((train_labels,labelsd),0)
print(train_images.shape)
print(train_labels.shape)

torch.Size([16250, 3, 224, 224])
torch.Size([16250])


In [8]:
train_images.dtype, train_labels.dtype, test_images.dtype, test_labels.dtype


(torch.float32, torch.float32, torch.float32, torch.uint8)

In [9]:
train_labels = train_labels.type(torch.uint8)
test_labels = test_labels.type(torch.uint8)

In [10]:
train_images.dtype, train_labels.dtype, test_images.dtype, test_labels.dtype


(torch.float32, torch.uint8, torch.float32, torch.uint8)

In [11]:
# loaded_data_l = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/lucknow_positive_images.pt")
# loaded_data_l.shape
# # positive_labels = torch.ones(len(loaded_data_l))
# # images_l = torch.cat((train_images,loaded_data_l))
# # labels_l = torch.cat((train_labels,positive_labels))
# # print(images_l.shape)
# # print(labels_l.shape)

In [12]:
# images.dtype
# labels_l = labels_l.to(torch.uint8)
# labels_l.dtype


In [13]:
train_labels_count = (train_labels == 1).sum()
test_labels_count = (test_labels == 1).sum()

print("Number of train labels with value 1:", train_labels_count)
print("Number of test labels with value 1:", test_labels_count)
# test_images=transform_test(test_images)

Number of train labels with value 1: tensor(1958)
Number of test labels with value 1: tensor(521)


### Data preporcessing for how much percentage of the data is used for finetune

In [14]:
# train_images=transform_eval(train_images)


In [15]:
# print(train_images.shape)
# print(test_images.shape)
train_set=torch.utils.data.TensorDataset(train_images,train_labels)
test_set=torch.utils.data.TensorDataset(test_images,test_labels)
train_loader=torch.utils.data.DataLoader(train_set,batch_size=512,shuffle=True,num_workers=8)
test_loader=torch.utils.data.DataLoader(test_set,batch_size=512,shuffle=False,num_workers=8)

In [16]:
len(train_set)

16250

In [18]:
base_encoder=torchvision.models.efficientnet_b0(pretrained=False)
# base_encoder
base_encoder.classifier[1]=nn.Linear(1280,2)
base_encoder.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

### fine-tune the downstram model  and predict on test data

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(base_encoder.parameters(), lr=0.001)
train_losses = []
train_accuracies = []
train_precisions = []
train_recalls = []
train_f1_scores = []
test_f1_scores = []
test_losses = []
test_accuracies = []
test_precisions = []
test_recalls = []
for epoch in range(30):
    base_encoder.train()
    true_labels = []
    pred_labels = []        
    running_loss = 0.0
    for i ,(inputs, labels) in enumerate(train_loader):
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        optimizer.zero_grad()
        output = base_encoder(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(output, 1)
        true_labels.extend(labels.data.cpu().numpy())
        pred_labels.extend(preds.data.cpu().numpy())
    epoch_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_loss)
    train_accuracy = accuracy_score(true_labels, pred_labels)
    train_accuracies.append(train_accuracy)
    train_precision = precision_score(true_labels, pred_labels)
    train_precisions.append(train_precision)
    train_recall = recall_score(true_labels, pred_labels)
    train_recalls.append(train_recall)
    train_f1 = f1_score(true_labels, pred_labels)
    train_f1_scores.append(train_f1)
    print(f'Epoch {epoch}, Loss: {epoch_loss}, Accuracy: {train_accuracy}, Precision: {train_precision}, Recall: {train_recall}, F1: {train_f1}')

    with torch.no_grad():
        base_encoder.eval()
        true_labels = []
        pred_labels = []
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to('cuda')
            labels = labels.to('cuda')
            output = base_encoder(inputs)
            _, preds = torch.max(output, 1)
            true_labels.extend(labels.data.cpu().numpy())
            pred_labels.extend(preds.data.cpu().numpy())
    test_accuracy = accuracy_score(true_labels, pred_labels)
    test_accuracies.append(test_accuracy)
    test_precision = precision_score(true_labels, pred_labels)
    test_precisions.append(test_precision)
    test_recall = recall_score(true_labels, pred_labels)
    test_recalls.append(test_recall)
    test_f1 = f1_score(true_labels, pred_labels)
    test_f1_scores.append(test_f1)
    print(f'Test - Accuracy: {test_accuracy}, Precision: {test_precision}, Recall: {test_recall}, F1: {test_f1}')

        


Epoch 0, Loss: 0.31909503628657415, Accuracy: 0.8760615384615384, Precision: 0.4830508474576271, Recall: 0.40755873340143, F1: 0.4421052631578947
Test - Accuracy: 0.8842222222222222, Precision: 0.0, Recall: 0.0, F1: 0.0


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1, Loss: 0.1685726140297376, Accuracy: 0.9393846153846154, Precision: 0.8120590121872996, Recall: 0.6465781409601634, F1: 0.7199317600227466
Test - Accuracy: 0.8844444444444445, Precision: 1.0, Recall: 0.0019193857965451055, F1: 0.0038314176245210726
Epoch 2, Loss: 0.14170619396613193, Accuracy: 0.9501538461538461, Precision: 0.8517156862745098, Recall: 0.7099080694586313, F1: 0.7743732590529249
Test - Accuracy: 0.9066666666666666, Precision: 0.8633093525179856, Recall: 0.23032629558541268, F1: 0.3636363636363637
Epoch 3, Loss: 0.10762520931317256, Accuracy: 0.9620307692307692, Precision: 0.8790276992651216, Recall: 0.7941777323799796, F1: 0.8344513013147303
Test - Accuracy: 0.9068888888888889, Precision: 0.9811320754716981, Recall: 0.19961612284069097, F1: 0.3317384370015949
Epoch 4, Loss: 0.08458990667508198, Accuracy: 0.971323076923077, Precision: 0.9172259507829977, Recall: 0.8375893769152196, F1: 0.8756006406833957
Test - Accuracy: 0.8975555555555556, Precision: 0.5406504065

In [34]:
result_dict = {}

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

percent = "100+25%"
ssl = "SimCLR"
with torch.no_grad():    
    base_encoder.eval()
    correct = 0
    total = 0
    y_pred = []
    y_test = []
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            output = base_encoder(inputs)
            _, preds = torch.max(output, 1)
            total += targets.size(0)
            correct += preds.eq(targets).sum().item()
            y_pred.extend(preds.cpu().numpy())
            y_test.extend(targets.cpu().numpy())
    
    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    
    print('Test Acc: %.3f%% (%d/%d)' % (100.*correct/total, correct, total))
    print(len(y_pred), len(y_test))
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)
    
    # If needed, class-specific metrics
    class_accuracy = cm.diagonal() / cm.sum(axis=1)
    class_precision = precision_score(y_test, y_pred, average=None)
    class_recall = recall_score(y_test, y_pred, average=None)
    class_f1 = f1_score(y_test, y_pred, average=None)

    print("Class-specific Accuracy:", class_accuracy)
    print("Class-specific Precision:", class_precision)
    print("Class-specific Recall:", class_recall)
    print("Class-specific F1 Score:", class_f1)
    result_dict[f"{percent} {ssl}"] = {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}
    print(result_dict)

Test Acc: 93.556% (4210/4500)
4500 4500
Accuracy:  0.9355555555555556
Precision:  0.8407079646017699
Recall:  0.5470249520153551
F1 Score:  0.6627906976744186
Confusion Matrix:
[[3925   54]
 [ 236  285]]
Class-specific Accuracy: [0.98642875 0.54702495]
Class-specific Precision: [0.94328286 0.84070796]
Class-specific Recall: [0.98642875 0.54702495]
Class-specific F1 Score: [0.96437346 0.6627907 ]
{'100+25% SimCLR': {'accuracy': 0.9355555555555556, 'f1': 0.6627906976744186, 'precision': 0.8407079646017699, 'recall': 0.5470249520153551}}


In [36]:
# y_pred,ytest=prdict(net_eval, test_loader, "30%", "ssl", device,)

In [37]:
import pandas as pd

In [38]:
print(result_dict)

{'100+25% SimCLR': {'accuracy': 0.9355555555555556, 'f1': 0.6627906976744186, 'precision': 0.8407079646017699, 'recall': 0.5470249520153551}}


In [39]:
import pickle

file_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/SSL_Experiments/SimCLR_experiments/siclr_result/100+25%_no_ssl.pkl"
with open(file_path, "wb") as file:
    pickle.dump(result_dict, file)

In [26]:
# import pickle

# # Load the dictionaries from the files
# with open("dictionary1_ssl.pkl", "rb") as file1, open("merged_dictionary_main.pkl", "rb") as file2:
#     dict1 = pickle.load(file1)
#     dict2 = pickle.load(file2)

# # Merge the dictionaries
# merged_dict = {**dict1, **dict2}

# # Save the merged dictionary to a file
# with open("merged_dictionary_main.pkl", "wb") as file:
#     pickle.dump(merged_dict, file)


In [27]:
# # import pickle

# # Load the merged dictionary from the file
# with open("merged_dictionary_main.pkl", "rb") as file:
#     merged_dict = pickle.load(file)

# # Print the merged dictionary
# print(merged_dict)


In [28]:
# # df=pd.read_csv("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/SSL_Experiments/patchfill_experiments/SSL_Result.csv")
# # df
# # # resdf = pd.DataFrame(result_dict)
# resdf = pd.DataFrame(merged_dict)
# resdf



In [29]:
# #list to tensor
# y_pred = torch.tensor(y_pred)
# y_test = torch.tensor(y_test)
# def results(y_labels,y_pred,plot_confusion = False):
#     y_labels = y_labels.to(device) 
#     y_pred = y_pred.to(device)
#     print(f"Accuracy = {(y_labels == y_pred).float().mean()}")
#     if plot_confusion:
#         cm = confusion_matrix(y_labels.cpu(), y_pred.cpu())
#         unique_labels = np.unique(y_labels.cpu())
#         cm_display = ConfusionMatrixDisplay(cm, display_labels=unique_labels).plot(values_format='d', cmap='Blues')
#         plt.show()

In [30]:
# results(y_test,y_pred,plot_confusion = True)

In [31]:
# import pandas as pd

# # Assuming df is your DataFrame
# df = pd.read_csv("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/SSL_Experiments/patchfill_experiments/SSL_Result.csv")

# # Extract the '100% SSL' column and convert it to a dictionary
# ssl_dict = df.set_index('Results')['100% SSL'].to_dict()

# # Display the created dictionary
# ssl_dict={'100% ssl':{'accuracy': 0.913217, 'f1': 0.696559, 'precision': 0.797193, 'recall': 0.654645}}


In [32]:
# file_path = "dictionary2.pkl"

# # Open the file in write mode
# with open(file_path, "wb") as file:
#     # Write the dictionary to the file
#     pickle.dump(ssl_dict, file)

In [33]:
# import pickle

# # Load the dictionaries from the files
# with open("merged_dictionary.pkl", "rb") as file1, open("dictionary2.pkl", "rb") as file2:
#     dict1 = pickle.load(file1)
#     dict2 = pickle.load(file2)

# # Merge the dictionaries
# merged_dict = {**dict1, **dict2}

# # Save the merged dictionary to a file
# with open("merged_dictionary.pkl", "wb") as file:
#     pickle.dump(merged_dict, file)

In [34]:
# resdf = pd.DataFrame(merged_dict)
# resdf


: 